In [34]:
from glob import glob
import numpy as np
import os
import pandas as pd
from PIL import Image
from skimage.io import imread
import warnings
warnings.filterwarnings(action='ignore')
Image.MAX_IMAGE_PIXELS = None
os.chdir ('/Users/kim/Data/CRUK-OHSU_onedrive final 2022-09-30/Manuscript/IMC-IF method paper/Cell_report_method_resource/github')

print('library import done')

library import done


In [35]:
# masked area save 

def makeareamask(name): 
    if (name == 'DE18.NE')|(name == 'DE28.NE.A') : 
        mask_ne = imread(glob(f'Data/IMC EMR (1)/*{name}*/5. Area mask/*_NE_Mask.tif')[0])

        mask_nondys = np.zeros(mask_ne.shape)
        mask_dys = np.zeros(mask_ne.shape)
        mask_imc = np.zeros(mask_ne.shape)

        mask_ne = np.where (mask_ne[:,:,1] !=0, 0, mask_ne[:,:,0])
        mask_nondys = np.where (mask_nondys[:,:,1] !=0, 0, mask_nondys[:,:,0])
        mask_dys = np.where (mask_dys[:,:,1] !=0, 0, mask_dys[:,:,0])
        mask_imc = np.where (mask_imc[:,:,1] !=0, 0, mask_imc[:,:,0])

    elif (name== 'PS14.24197-1E')| ( name == 'PS15.19650-B3') : 
        mask_nondys = imread(glob(f'Data/IMC EMR (1)/*{name}*/5. Area mask/*_NonDys_Mask.tif')[0]) 
        mask_dys = imread(glob(f'Data/IMC EMR (1)/*{name}*/5. Area mask/*_Dys_Mask.tif')[0])
        mask_imc = imread(glob(f'Data/IMC EMR (1)/*{name}*/5. Area mask/*_IMC_Mask.tif')[0])
        mask_ne = np.zeros(mask_nondys.shape) 

        mask_ne = np.where (mask_ne[:,:,1] !=0, 0, mask_ne[:,:,0])
        mask_nondys = np.where (mask_nondys[:,:,1] !=0, 0, mask_nondys[:,:,0])
        mask_dys = np.where (mask_dys[:,:,1] !=0, 0, mask_dys[:,:,0])
        mask_imc = np.where (mask_imc[:,:,1] !=0, 0, mask_imc[:,:,0]) 

    elif name == 'PS16.38634-F2':
        pt=name.split('.')[0]
        mask_nondys = imread(glob(f'Data/IMC EMR (1)/*{pt}*/5. Area mask/*_NonDys*')[0])  
        mask_ne = np.zeros(mask_nondys.shape) 
        mask_dys = np.zeros(mask_nondys.shape) 
        mask_imc = np.zeros(mask_nondys.shape) 


    mask_ne_re =np.array(mask_ne,np.uint8)
    Image.fromarray(mask_ne_re).save(f'Analysis/new_July_2021/areamask/{name}_mask_ne.tiff')

    mask_nondys_re =np.array(mask_nondys,np.uint8)
    Image.fromarray(mask_nondys_re).save(f'Analysis/new_July_2021/areamask/{name}_mask_nondys.tiff')

    mask_dys_re =np.array(mask_dys,np.uint8)
    Image.fromarray(mask_dys_re).save(f'Analysis/new_July_2021/areamask/{name}_mask_dys.tiff')


    mask_imc_re =np.array(mask_imc,np.uint8)
    Image.fromarray(mask_imc_re).save(f'Analysis/new_July_2021/areamask/{name}_mask_imc.tiff')

    print(name, 'done')

In [36]:
# Select cells from Dataframe : area masks among nuc or cell masks 

def masked_df(name, cat, areamask): 
    pt=name.split('.')[0]
    # Cell mask 
    if cat == 'IF': 
        path = 'Analysis/new_July_2021/ncratio_feature/merge_IF.csv'
    elif cat == 'IMC':
        path = 'Analysis/new_July_2021/ncratio_feature/merge_IMC.csv'

    df=pd.read_csv(path)

    # Read Nuc/cell masks 
    IF_nuc = imread(f'Analysis/new_July_2021/maskmerge_mesmer_nuc_0325/{name}.tiff') 
    IF_cell = imread(f'Analysis/new_July_2021/maskmerge_mesmer_0325/{name}.tiff') 

    IMC_cell = imread(f'Analysis/new_July_2021/IMC_mesmer/{pt}.tiff') 
    IMC_nuc = imread(f'Analysis/new_July_2021/IMC_mesmer_nuc/{pt}.tiff')

    IMC_rim = np.where(IMC_nuc != 0, 0, IMC_cell)
    IF_rim = np.where(IF_nuc != 0, 0, IF_cell)

    IF_nuc_final = np.where(IF_rim !=0, 0, IF_cell)
    IMC_nuc_final = np.where(IMC_rim !=0, 0, IMC_cell)

    # Find Cell ID 
    df_ID = df[df['name']== name]
    # maks
    if cat == 'IF': 
        ptmask= np.where(areamask==255,IF_cell,0)
    if cat == 'IMC': 
        ptmask= np.where(areamask==255,IMC_cell,0)
    
    ptmask=ptmask.flatten()
    ptmask=df_ID['cell_id'].isin(ptmask)
    finaldf=df_ID[ptmask]
    return finaldf 

In [37]:
# Merge dataframe for three diseae status masks 

def merge_ds_mask(name, cat):
    # Disease mask 
    mask_ne_final=imread(f'Analysis/new_July_2021/areamask/{name}_mask_ne.tiff')
    mask_nondys_final=imread(f'Analysis/new_July_2021/areamask/{name}_mask_nondys.tiff')
    mask_dys_final=imread(f'Analysis/new_July_2021/areamask/{name}_mask_dys.tiff')
    mask_imc_final=imread(f'Analysis/new_July_2021/areamask/{name}_mask_imc.tiff')
    df_ne= masked_df(name=name, cat=cat, areamask=mask_ne_final)
    df_ne['dx']='ne'
    df_nondys = masked_df(name=name, cat=cat, areamask=mask_nondys_final)
    df_nondys['dx'] = 'nondys'
    df_dys = masked_df(name=name, cat=cat, areamask=mask_dys_final)
    df_dys['dx'] = 'dys'
    df_imc = masked_df(name=name, cat=cat, areamask=mask_imc_final)
    df_imc['dx'] = 'imc'
    df_final=pd.concat([df_ne, df_nondys,df_dys, df_imc], axis=0)
    df_final.to_csv(f'Analysis/new_July_2021/ncratio_feature/dxmask/{cat}_{name}_dxmask.csv')
    return df_final

In [33]:
names = ['DE18.NE', 'DE28.NE.A', 'PS14.24197-1E', 'PS15.19650-B3', 'PS16.38634-F2']

for name in names:
    makeareamask(name=name)
print('finally done')

DE18.NE done
DE28.NE.A done
PS14.24197-1E done
PS15.19650-B3 done
PS16.38634-F2 done
finally done


In [10]:
cat = 'IF'

for name in names: 
    merge_ds_mask(name, cat='IF') # cat='IMC'
    print(name, 'done')
print('finally done')

file_list= sorted(glob(f'Analysis/new_July_2021/ncratio_feature/dxmask/{cat}_*'))
df_mask_column =pd.read_csv(file_list[0])
column_names=df_mask_column.columns
df_merg = pd.DataFrame(columns = column_names)

for name in names:
    df_next = pd.read_csv(f'Analysis/new_July_2021/ncratio_feature/dxmask/{cat}_{name}_dxmask.csv')
    df_merg = pd.concat([df_merg,df_next], ignore_index=True)

df_merg.to_csv(f'Analysis/new_July_2021/ncratio_feature/merge_maskdx_{cat}.csv')
print('done')


done
done
done
done
DE18.NE done
done
done
done
done
DE28.NE.A done
done
done
done
done
PS14.24197-1E done
done
done
done
done
PS15.19650-B3 done
done
done
done
done
PS16.38634-F2 done
finally done
done


In [13]:
cat = 'IMC'

for name in names: 
    merge_ds_mask(name, cat='IMC') # cat='IMC'
    print(name, 'done')
print('finally done')

file_list= sorted(glob(f'Analysis/new_July_2021/ncratio_feature/dxmask/{cat}_*'))
df_mask_column =pd.read_csv(file_list[0])
column_names=df_mask_column.columns
df_merg = pd.DataFrame(columns = column_names)

for name in names:
    df_next = pd.read_csv(f'Analysis/new_July_2021/ncratio_feature/dxmask/{cat}_{name}_dxmask.csv')
    df_merg = pd.concat([df_merg,df_next], ignore_index=True)

df_merg.to_csv(f'Analysis/new_July_2021/ncratio_feature/merge_maskdx_{cat}.csv')
print('done') 

done
done
done
done
DE18.NE done
done
done
done
done
DE28.NE.A done
done
done
done
done
PS14.24197-1E done
done
done
done
done
PS15.19650-B3 done
done
done
done
done
PS16.38634-F2 done
finally done
done
